In [1]:
import pandas as pd

In [5]:
uri = '/Users/david.hedges/Documents/woodahl_lab/pgx_tool/pgx/Star Allele Dictionary.xlsx'
df = pd.read_excel(uri, na_filter=None)

In [6]:
df = df.replace({'-':''}, regex=True)

changes = ['change_1','change_2','change_3','change_4','change_5',
           'change_6','change_7','change_8','change_9','change_10',
           'change_11','change_12','change_13','change_14','change_15',
           'change_16','change_17','change_18','change_19','change_20',
           'change_21','change_22','change_23','change_24','change_25',
           'change_26','change_27','change_28','change_29','change_30',
           'change_31','change_32','change_33','change_34','change_35',
           'change_36','change_37','change_38']

for i in changes:
    df[i] = df[i].str.replace(r"\(.*\)","")
    
df

,Allele,mtID,change_1,change_2,change_3,change_4,change_5,change_6,change_7,change_8,...,change_29,change_30,change_31,change_32,change_33,change_34,change_35,change_36,change_37,change_38
0,CYP2A13*1.001,,,,,,,,,,...,,,,,,,,,,
1,CYP2A13*1.002,,12C>A,,,,,,,,...,,,,,,,,,,
2,CYP2A13*1.003,,187T>C,,,,,,,,...,,,,,,,,,,
3,CYP2A13*1.004,,1757A>G,,,,,,,,...,,,,,,,,,,
4,CYP2A13*1.005,,1894G>T,,,,,,,,...,,,,,,,,,,
5,CYP2A13*1.006,,6461T>A,,,,,,,,...,,,,,,,,,,
6,CYP2A13*1.007,,7571G>C,,,,,,,,...,,,,,,,,,,
7,CYP2A13*1.008,,1479T>C,1240A>G,411G>A,1757A>G,2366C>T,6424C>T,6432C>T,7233T>G,...,,,,,,,,,,
8,CYP2A13*1.009,,523C>T,,,,,,,,...,,,,,,,,,,
9,CYP2A13*1.010,,1991C>A,,,,,,,,...,,,,,,,,,,


Need to convert transcript location to genomic location

make it compatible with both build 37 and 38

Plan:
- use try/except to parse the 12C>A (or 1634_1635insACC or substitution) type string into integers and the bases.
- regular expressions!!
- match the start from the transcript location to the genomic location
- wave magic wand and get the mtIDs for each variant (not just each haplotype)
- correlate each haplotype with an mtID profile

In [8]:
genes38 = {
    'CYP2A13':41088451,
    'CYP2C8':95036772,
    'CYP2C9':94938658,
    'CYP2C19':94762681,
    'CYP2D6':42126499,
    'CYP2F1':41114292,
    'CYP2J2':59893308,
    'CYP2R1':14878005,
    'CYP2S1':41193219,
    'CYP2W1':983155,
    'CYP3A7':99705036,
    'CYP3A43':99827626,
    'CYP4F2':15878023,
    'CYP26A1':93073475,
    'NUDT14':105172939
}

genes37 = {
    'CYP2A13':41594356,
    'CYP2C8':96796529,
    'CYP2C9':96698350,
    'CYP2C19':96522463,
    'CYP2D6':42522501,
    'CYP2F1':41620312,
    'CYP2J2':60358980,
    'CYP2R1':14899551,
    'CYP2S1':41699115,
    'CYP2W1':1020100,
    'CYP3A7':99302660,
    'CYP3A43':99425636,
    'CYP4F2':15988834,
    'CYP26A1':94833232,
    'NUDT14':105639274
}

chroms = {
    'CYP2A13':41594356,
    'CYP2C8':96796529,
    'CYP2C9':96698350,
    'CYP2C19':10,
    'CYP2D6':22,
    'CYP2F1':19,
    'CYP2J2':1,
    'CYP2R1':11,
    'CYP2S1':19,
    'CYP2W1':7,
    'CYP3A7':7,
    'CYP3A43':7,
    'CYP4F2':19,
    'CYP26A1':10,
    'NUDT14':14
}